Important links : https://docs.dask.org/en/stable/dataframe.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
generated_data = 'generated_data/'
parquet_file_path = generated_data +  "speaker_attributes.parquet"

### Loading the parquet file

In [3]:
%time parquet_df = pd.read_parquet(parquet_file_path)

Wall time: 18.9 s


In [4]:
%time parquet_df.head()

Wall time: 0 ns


,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [5]:
%time parquet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9055981 entries, 0 to 9055980
Data columns (total 15 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   aliases             object
 1   date_of_birth       object
 2   nationality         object
 3   gender              object
 4   lastrevid           int64 
 5   ethnic_group        object
 6   US_congress_bio_ID  object
 7   occupation          object
 8   party               object
 9   academic_degree     object
 10  id                  object
 11  label               object
 12  candidacy           object
 13  type                object
 14  religion            object
dtypes: int64(1), object(14)
memory usage: 1.0+ GB
Wall time: 39 ms


In [ ]:
%time mem_usage = parquet_df.memory_usage(deep=True).sum()

In [ ]:
print('the parquet dataframe takes about {:.2f} gigabytes in memory'.format(mem_usage*1e-9 ))

### Loading the quote sample file

In [ ]:
%time quotebank_2019_sample = pd.read_json('generated_data/quotes-2019-nytimes.json.bz2', lines=True, compression='bz2')
%time quotebank_2019_sample.head()

In [ ]:
%time mem_usage2 = quotebank_2019_sample.memory_usage(deep=True).sum()
print('the quotebank sample dataframe takes about {:.2f} megabytes in memory'.format(mem_usage2*1e-6 ))

Selecting only the first qid in qids column of sample data and dropping rows with no qids

In [ ]:
def preprocess(df):
    # remove lines with empty qids
    df = df[~df['qids'].str.len().eq(0)]
    # take only the first qid
    df['qids'] = df['qids'].apply(lambda x: x[0]) # this gives a warning
    return df

In [ ]:
quotebank_2019_sample_clean = preprocess(quotebank_2019_sample)
quotebank_2019_sample_clean.head()

### Merging the parquet file and the qid samples

In [ ]:
%time merged_df = quotebank_2019_sample_clean.merge(parquet_df, left_on = 'qids', right_on = 'id')

In [ ]:
merged_df.head()

In [ ]:
%time mem_usage3 = merged_df.memory_usage(deep=True).sum()*1e-6
print('the quotebank merged dataframe takes about {:.2f} megabytes in memory'.format(mem_usage3))

### Other stuff

TODO : make the labels from the parquet file readable using the QID code from the QID to label notebook

In [ ]:
type(merged_df['occupation'][1])

In [ ]:
merged_df.dtypes

Just looking at the types of every column of the new dataframe

In [ ]:
obj_types = {col: set(map(type, merged_df[col])) for col in merged_df.select_dtypes(include=[object])}

print(obj_types)

In [ ]:
merged_df.iloc[1]

A lot of the columns contain multiple QID entries : 

In [ ]:
print('maximum number of QID entry for each of the new columns')
print('nationality :', merged_df['nationality'].apply(lambda x : len(x) if x is not None else 0).max())
print('gender :', merged_df['gender'].apply(lambda x : len(x) if x is not None else 0).max())
print('ethnic_group :', merged_df['ethnic_group'].apply(lambda x : len(x) if x is not None else 0).max())
print('US_congress_bio_ID :', merged_df['US_congress_bio_ID'].apply(lambda x : len(x) if x is not None else 0).max())
print('occupation :', merged_df['occupation'].apply(lambda x : len(x) if x is not None else 0).max())
print('party :', merged_df['party'].apply(lambda x : len(x) if x is not None else 0).max())
print('academic_degree :', merged_df['academic_degree'].apply(lambda x : len(x) if x is not None else 0).max())
print('id :', merged_df['id'].apply(lambda x : len(x) if x is not None else 0).max())
print('label :', merged_df['label'].apply(lambda x : len(x) if x is not None else 0).max())
print('candidacy :', merged_df['candidacy'].apply(lambda x : len(x) if x is not None else 0).max())
print('type :', merged_df['type'].apply(lambda x : len(x) if x is not None else 0).max())
print('religion :', merged_df['religion'].apply(lambda x : len(x) if x is not None else 0).max())

Since this time it is a reasonable number ( vers 451 in the 2019 sample dataset) we could potentially match all entries to terms inside of 

Instead of manually checking we use library re to scan every column if there are QIDs. 
Most columns are composed of a numpy nd.array containing a tuple of strings, every string being a qid. So we take the first access the tuple, then take the first element, then check if it's a QID.

In [ ]:
import re

In [ ]:
def check_if_qid(df):
    '''
    take first term of list out of ndarray, and checks if it's a QID
    '''
    return df.apply(lambda x: x[0] if x is not None else None).str.contains(r'[Q][0-9]+').any()

In [ ]:
print('column : nationality ', check_if_qid(merged_df['nationality']))
print('column : gender ', check_if_qid(merged_df['gender']))
print('column : ethnic_group ', check_if_qid(merged_df['ethnic_group']))
print('column : US_congress_bio_ID ', check_if_qid(merged_df['US_congress_bio_ID']))
print('column : occupation ', check_if_qid(merged_df['occupation']))
print('column : party ', check_if_qid(merged_df['party']))
print('column : academic_degree ', check_if_qid(merged_df['academic_degree']))
print('column : id ', check_if_qid(merged_df['id']))
print('column : label ', check_if_qid(merged_df['label']))
print('column : candidacy ', check_if_qid(merged_df['candidacy']))
print('column : type ', check_if_qid(merged_df['type']))
print('column : religion ', check_if_qid(merged_df['religion']))

Selected columns that need QID to label translations : 
 - nationality
 - gender
 - ethnic_group
 - occupation
 - party
 - academic_degree
 - candidacy
 - religion

In [ ]:
def process_chunk(chunk, qids_clean_merged):
    """
    this function processes one chunk of data
    """
    qids_clean_merged = qids_clean_merged.reindex(columns=qids_clean_merged.columns.union(chunk.columns))
    
    qids_clean_merged.update(chunk)

    return qids_clean_merged

def process_qid_one(path_to_file, qids_clean, chunksize = 10 ** 4):
    """
    this function processes a csv file in chunks
    
    here it attributes qid to their respective labels and descriptions 
    ( by joining the qid label df to the df not containing them)
    """
    qids_clean_merged = qids_clean.copy()
    qids_clean_merged['Label']=np.NaN
    qids_clean_merged['Description']=np.NaN
    qids_clean_merged.set_index('qids',inplace=True)
    qids_clean_merged.index.rename('QID', inplace=True)
    
    with pd.read_csv(path_to_file, compression='bz2', index_col='QID', chunksize=chunksize) as df_reader:
        for chunk in df_reader:
            qids_clean_merged = process_chunk(chunk, qids_clean_merged)
    return qids_clean_merged

In [ ]:
qids_onlyquotebank_path = generated_data + "wikidata_labels_descriptions_quotebank.csv.bz2"

In [ ]:
def first_qid(df):
    '''
    take first term of list out of ndarray
    '''
    return df.apply(lambda x: x[0] if x is not None else None)

In [ ]:
first_qid(merged_df['religion']).dropna()

In [ ]:
def prep_df(df):
    df = first_qid(df).to_frame().copy()
    return df.rename(columns = {df.columns[0]:'qids'})

In [ ]:
nationality_df = prep_df(merged_df['nationality'])
gender_df = prep_df(merged_df['gender'])
ethnic_group_df = prep_df(merged_df['ethnic_group'])
occupation_df = prep_df(merged_df['occupation'])
party_df = prep_df(merged_df['party'])
academic_degree_df = prep_df(merged_df['academic_degree'])
candidacy_df = prep_df(merged_df['candidacy'])
religion_df = prep_df(merged_df['religion'])
religion_df

In [ ]:
%time nationality_df = process_qid_one(qids_onlyquotebank_path, nationality_df , chunksize = 10 ** 6)
%time gender_df = process_qid_one(qids_onlyquotebank_path, gender_df , chunksize = 10 ** 6)
%time ethnic_group_df = process_qid_one(qids_onlyquotebank_path, ethnic_group_df , chunksize = 10 ** 6)
%time occupation_df = process_qid_one(qids_onlyquotebank_path, occupation_df , chunksize = 10 ** 6)
%time party_df = process_qid_one(qids_onlyquotebank_path, party_df , chunksize = 10 ** 6)
%time academic_degree_df = process_qid_one(qids_onlyquotebank_path, academic_degree_df , chunksize = 10 ** 6)
%time candidacy_df = process_qid_one(qids_onlyquotebank_path, candidacy_df , chunksize = 10 ** 6)
%time religion_df = process_qid_one(qids_onlyquotebank_path, religion_df , chunksize = 10 ** 6)

In [ ]:
religion.dropna()